<a href="https://colab.research.google.com/github/Ramubala/hindi-tokenizer/blob/main/Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle transformers[torch]
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d vaibhavkumar11/hindi-english-parallel-corpus
!mkdir train
!unzip hindi-english-parallel-corpus.zip -d data

hindi-english-parallel-corpus.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘train’: File exists
Archive:  hindi-english-parallel-corpus.zip
replace data/hindi_english_parallel.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/hindi_english_parallel.csv  


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [ ]:
data = pd.read_csv('data/hindi_english_parallel.csv')
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True)

languages = ['hindi','english']
normalizable_language = ['english']

# convert the langauage columns to string dtype
for col in languages:
  data[col] = data[col].astype(str).astype(pd.StringDtype()) 

data.head()

,index,hindi,english
0,0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [ ]:
data.dtypes, data.shape

(index       int64
 hindi      string
 english    string
 dtype: object, (1358340, 3))

In [ ]:
def get_training_corpus(col):
    for i in range(0, len(data), 1000):
        yield data[i : i + 1000][col]

In [ ]:
for col in languages:
  print(col)
  tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

  # no normalizer is needed for hindi language
  if col in normalizable_language:
    tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
    )
  else:
    pass

  # pre tokenize using whitespaces
  tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

  # use the models to get subword tokenization
  special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
  trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens, min_frequency=5)

  tokenizer.train_from_iterator(get_training_corpus(col), trainer=trainer)

  cls_token_id = tokenizer.token_to_id("[CLS]")
  sep_token_id = tokenizer.token_to_id("[SEP]")

  # to add separator for sentences
  tokenizer.post_processor = processors.TemplateProcessing(
      single=f"[CLS]:0 $A:0 [SEP]:0",
      pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
      special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
  )

  # viewing the tokenized sentences
  for i in range(10):
    encoding = tokenizer.encode(data.loc[i][col])
    print(data.loc[i][col])
    print(encoding.tokens)

  tokenizer.save("{}_tokenizer.json".format(col))

hindi
अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
['[CLS]', 'अपने', 'अनुप्रयोग', 'को', 'पहुंच', '##नीयता', 'व्यायाम', 'का', 'लाभ', 'दें', '[SEP]']
एक्सेर्साइसर पहुंचनीयता अन्वेषक
['[CLS]', 'एक्से', '##र्स', '##ाइस', '##र', 'पहुंच', '##नीयता', 'अन्वेषक', '[SEP]']
निचले पटल के लिए डिफोल्ट प्लग-इन खाका
['[CLS]', 'निचले', 'पटल', 'के', 'लिए', 'डिफ', '##ोल्ट', 'प्लग', '-', 'इन', 'खाका', '[SEP]']
ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
['[CLS]', 'ऊपरी', 'पटल', 'के', 'लिए', 'डिफ', '##ोल्ट', 'प्लग', '-', 'इन', 'खाका', '[SEP]']
उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
['[CLS]', 'उन', 'प्लग', '-', 'इन', '##ों', 'की', 'सूची', 'जिन्हें', 'डिफ', '##ोल्ट', 'रूप', 'से', 'निष्क्रिय', 'किया', 'गया', 'है', '[SEP]']
अवधि को हाइलाइट रकें
['[CLS]', 'अवधि', 'को', 'हाइ', '##लाइट', 'रक', '##ें', '[SEP]']
पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि
['[CLS]', 'पहुंच', '##नीय', 'आस', '##ंधि', '(', 'नोड', ')', 'को', 'चुनते', 'समय', 'हाइ', '##लाइट', 'बक्से', 'की', 'अवधि', '[SEP